<a href="https://colab.research.google.com/github/ZahraDehghani99/FacemaskWearingAlertSystem/blob/main/prepare_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare dataset

In [ ]:
!pip install tsai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 241 kB 5.2 MB/s 
     |████████████████████████████████| 2.5 MB 38.0 MB/s 
     |████████████████████████████████| 750.6 MB 6.6 kB/s 
     |████████████████████████████████| 19.1 MB 42.2 MB/s 
     |████████████████████████████████| 19.1 MB 24.9 MB/s 
     |████████████████████████████████| 21.0 MB 27.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0+cu113
    Uninstalling torchvision-0.13.0+cu113:
      Successfully uninstalled torchvision-0.13.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.1

In [ ]:
import os
import random
import shutil
import numpy as np
from tqdm import tqdm
from sys import getsizeof
from tsai.all import *

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from imutils import paths

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## for unmount drive
# from google.colab import drive
# drive.flush_and_unmount()

## RMFD 

### Make data balance

در مقاله ذکر شده که این مجموعه دادگان شامل ۵۰۰۰ عکس باماسک و ۵۰۰۰ عکس بدون ماسک است. اما با مراجعه به صفحه ی گیت هاب مربوط به مقاله ی این دادگان، انواع مختلفی از دیتاست را مشاهده می کنیم که در برخی از آن ها تصاویر با ماسک از تصاویر بدون ماسک جدا نشده اند. تنها داده ای که نزدیک به اظهارات مقاله است [این](https://drive.google.com/file/d/1UlOk6EtiaXTHylRUx2mySgvJX9ycoeBp/view) لینک می باشد.

طبق توضیحات گیت هاب این پوشه شامل ۵۰۰۰ عکس انسان های مختلف با ماسک و ۹۰۰۰۰ عکس ار انسان های مختلف بدون ماسک است. پس از دانلود این مجموعه دادگان متوجه می شویم برخلاف 
چیزی که در گیت هاب دادگان ذکر شده بود تنها ۲۲۰۳ عکس با ماسک و ۹۰۴۶۸ عکس بدون ماسک داریم. برای اینکه مانند مقاله کلاس بندی متوازنی داشته باشیم تمام  داده های با ماسک را استفاده می کنیم و به صورت رندوم ۲۲۰۳ عکس نیز ار داده های بدون ماسک انتخاب می کنیم. 

In [ ]:
RMFD_PATH = "/content/drive/MyDrive/DIP_final_proj/face-mask-classifier-dataset/RMFD/"

#### Read with-mask images

In [ ]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

imagePaths = list(paths.list_images(RMFD_PATH + "AFDB_masked_face_dataset"))

print(f"[INFO] loading finished. {len(imagePaths)} images loaded.")

[INFO] loading images...
[INFO] loading finished. 2203 images loaded.


#### Read without-mask images

حال داده های موجود در مسیر داده های بدون ماسک را می خوانیم تا به صورت رندوم ۲۲۰۳ تا از آن ها را انتخاب کنیم.

In [ ]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

imagePaths = list(paths.list_images(RMFD_PATH + "AFDB_face_dataset"))

print(f"[INFO] loading finished. {len(imagePaths)} images loaded.")

[INFO] loading images...
[INFO] loading finished. 90468 images loaded.


##### Select 2203 random images and Creae new withou-mask images folder

In [ ]:
random_selected_face_dataset = random.sample(imagePaths, 2203)
print(f'Selected {len(random_selected_face_dataset)} images from AFDB_face_dataset')

Selected 2203 images from AFDB_face_dataset


حال باید عکس های انتخاب شده از مجموعه ی بدون ماسک را به یک فولدر دیگر انتقال دهیم تا از آن ها برای آموزش مدل استفاده شود.


اول یک فولدر به نام withou_mask در مسیر داده های آموزشی ایجاد می کنیم و سپس داده ها را به  آنجا انتقال می دهیم. 

در نهایت اسم پوشه ای که شامل عکس های با ماسک هست را نیز به with_mask تغییر می دهیم تا همه ی داده های ما برچسب یکسانی داشته باشند و در کلاس بندی به مشکل نخوریم.

In [ ]:
for image in tqdm(random_selected_face_dataset):
    shutil.copy(image, RMFD_PATH+"without_mask/")

100%|██████████| 2203/2203 [01:13<00:00, 29.92it/s]


حال پوشه ی شامل عکس های بدون ماسک را که خودمان ایجاد کردیم به همراه پوشه ی عکس های با ماسک را در یک مسیر جدید ذخیره می کنیم و از این به بعد از این داده های ویرایش شده استفاده می کنیم.


حالا می خواهیم داده ها مربوط به هر دو پوشه را بخوانیم و آن ها را به فرمت numpy ذخیره کنیم.

### Read whole dataset and save in `.npy` format

In [ ]:
# !unzip -q /content/drive/MyDrive/DIP_final_proj/face-mask-classifier-dataset/dataset_RMFD.zip -d /content/drive/MyDrive/DIP_final_proj/face-mask-classifier-dataset/dataset_RMFD

In [ ]:
TRAIN_DATA_DIR = "/content/drive/MyDrive/DIP_final_proj/face-mask-classifier-dataset/"

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(TRAIN_DATA_DIR + "dataset_RMFD/dataset_RMFD"))
print(f"[INFO] loading finished. {len(imagePaths)} images loaded.")

[INFO] loading images...
[INFO] loading finished. 4406 images loaded.


In [ ]:
data = []
labels = []

# loop over the image paths
for imagePath in tqdm(imagePaths):
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[8] # idx 8 because for `without_mask` label we have different directory

	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	# image = preprocess_input(image)

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

100%|██████████| 4406/4406 [00:25<00:00, 172.57it/s]


In [ ]:
# convert the data and labels to NumPy arrays
rmfd_data = np.array(data, dtype="float32")
rmfd_labels = np.array(labels)

print(f'rmfd data shape: {rmfd_data.shape}')
print(f'rmfd labels shape: {rmfd_labels.shape}')

rmfd data shape: (4406, 224, 224, 3)
rmfd labels shape: (4406,)


In [ ]:
print(f'rmfd data : {bytes2GB(getsizeof(rmfd_data))} GB')

rmfd data : 2.47 GB


In [ ]:
rmfd_data = rmfd_data / 255.0

In [ ]:
print(f'rmfd data : {bytes2GB(getsizeof(rmfd_data))} GB')

rmfd data : 2.47 GB


In [ ]:
np.save(f'{TRAIN_DATA_DIR}/rmfd_data.npy', rmfd_data)
np.save(f'{TRAIN_DATA_DIR}/rmfd_labels.npy', rmfd_labels)

## CDD

### Read whole dataset and save in .npy format

In [ ]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(TRAIN_DATA_DIR + "dataset_CDD"))
print(f"[INFO] loading finished. {len(imagePaths)} images loaded.")

In [ ]:
from tqdm import tqdm
data = []
labels = []

# loop over the image paths
for imagePath in tqdm(imagePaths):
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	# image = preprocess_input(image)

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

  4%|▍         | 177/4092 [00:57<01:58, 33.07it/s]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
100%|██████████| 4092/4092 [10:42<00:00,  6.37it/s]


In [ ]:
# convert the data and labels to NumPy arrays
cdd_data = np.array(data, dtype="float32")
cdd_labels = np.array(labels)

print(f'cdd data shape: {cdd_data.shape}')
print(f'cdd labels shape: {cdd_labels.shape}')

In [ ]:
np.save(f'{TRAIN_DATA_DIR}/cdd_data.npy', cdd_data)
np.save(f'{TRAIN_DATA_DIR}/cdd_labels.npy', cdd_labels)

## SMFD

### Read whole dataset and save in .npy format

In [ ]:
!unzip -q /content/drive/MyDrive/DIP_final_proj/face-mask-classifier-dataset/dataset_SMFD.zip -d /content/drive/MyDrive/DIP_final_proj/face-mask-classifier-dataset/dataset_SMFD

In [ ]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(TRAIN_DATA_DIR + "dataset_SMFD/dataset_SMFD"))
print(f"[INFO] loading finished. {len(imagePaths)} images loaded.")

[INFO] loading images...
[INFO] loading finished. 1376 images loaded.


In [ ]:
from tqdm import tqdm
data = []
labels = []

# loop over the image paths
for imagePath in tqdm(imagePaths):
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	# image = preprocess_input(image)

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

100%|██████████| 1376/1376 [00:09<00:00, 144.41it/s]


In [ ]:
# convert the data and labels to NumPy arrays
smfd_data = np.array(data, dtype="float32")
smfd_labels = np.array(labels)

print(f'smfd data shape: {smfd_data.shape}')
print(f'smfd labels shape: {smfd_labels.shape}')

smfd data shape: (1376, 224, 224, 3)
smfd labels shape: (1376,)


In [ ]:
np.save(f'{TRAIN_DATA_DIR}/smfd_data.npy', smfd_data)
np.save(f'{TRAIN_DATA_DIR}/smfd_labels.npy', smfd_labels)